In [1]:
import requests
from bs4 import BeautifulSoup
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Util.Padding import pad, unpad
from binascii import unhexlify
import base64
import json
import urllib.parse

In [2]:
userid="satvikmishra2615@gmail.com"
password="Maharajji@66"

In [3]:
with open("data_ie.json", "r") as json_file:
    inp = json.load(json_file)

In [4]:
url = "https://www.dgft.gov.in/CP/"
session = requests.Session()
response = session.get(url)


In [5]:
from bs4 import BeautifulSoup
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
if csrf_value := soup.find('meta', attrs={'name': '_csrf'}):
    csrf = csrf_value.get('content')
    print("CSRF Content:", csrf)
else:
    print("CSRF Meta tag not found.")

CSRF Content: 90e0b668-755a-43a1-935e-c0a8b3b8e435


In [6]:
url = f"https://www.dgft.gov.in/CP/webHP?requestType=ApplicationRH&actionVal=loginModel&screenId=90000512&_csrf={csrf}"
data = {
    "portal": "CAS"
}
response = session.post(url, data=data)
html = response.content
soup = BeautifulSoup(html, "html.parser")
desired_id_element = soup.find(id="temp_key")
salt = desired_id_element.get('value')

In [7]:
url = f"https://www.dgft.gov.in/CP/webHP?requestType=ApplicationRH&actionVal=commonCaptcha&screenId=90000512&_csrf={csrf}"
data = {
    "portal": "CAS"
}
response = session.post(url,  data=data)

In [8]:
url = f"https://www.dgft.gov.in/CP/webHP?requestType=ApplicationRH&actionVal=formElements&screenId=90000512&_csrf={csrf}"
data = {
    "isShow": "true",
    "personType": "-1"
}
response = session.post(url, data=data)

In [9]:
def gen_captcha():
    img_url ="https://www.dgft.gov.in/CP/SimpleCaptcha?1692453334489"
    response = session.get(img_url, stream=True)
    setcookie = response.headers
    if response.status_code == 200:
        image_path = "captcha_image2.jpg"
        with open(image_path, 'wb') as image_file:
            for chunk in response.iter_content(chunk_size=128):
                image_file.write(chunk)
        print("CAPTCHA image saved as:", image_path)
    else:
        print("Failed to fetch CAPTCHA image")
    return setcookie

In [10]:
class AesUtil:
    def __init__(self, key_size, iteration_count):
        self.key_size = key_size // 8
        self.iteration_count = iteration_count

    def generate_key(self, salt, pass_phrase):
        return PBKDF2(
            pass_phrase.encode(),
            unhexlify(salt),
            dkLen=self.key_size,
            count=self.iteration_count,
        )

    def encrypt(self, salt, iv, pass_phrase, plain_text):
        key = self.generate_key(salt, pass_phrase)
        cipher = AES.new(key, AES.MODE_CBC, iv=unhexlify(iv))
        padded_text = pad(plain_text.encode(), AES.block_size)
        ciphertext = cipher.encrypt(padded_text)
        return base64.b64encode(ciphertext).decode()

    def decrypt(self, salt, iv, pass_phrase, cipher_text):
        key = self.generate_key(salt, pass_phrase)
        cipher = AES.new(key, AES.MODE_CBC, iv=unhexlify(iv))
        ciphertext = base64.b64decode(cipher_text)
        decrypted_data = unpad(cipher.decrypt(ciphertext), AES.block_size)
        return decrypted_data.decode()

def encrypt_password(plain_text, salt, iv, pass_phrase):
    iteration_count = 1000
    key_size = 128
    aes_util = AesUtil(key_size, iteration_count)
    return aes_util.encrypt(salt, iv, pass_phrase, plain_text)

def decrypt_password(cipher_text, salt, iv, pass_phrase):
    iteration_count = 1000
    key_size = 128
    aes_util = AesUtil(key_size, iteration_count)
    return aes_util.decrypt(salt, iv, pass_phrase, cipher_text)
plain_text = password
iv = "4d67fd8ee80132c6115e39880b08165d"
pass_phrase = "tcs@1234"
encrypted_result = encrypt_password(plain_text, salt, iv, pass_phrase)
print("Encrypted Password:", encrypted_result)



Encrypted Password: jywM8USaGkUaoZwBgdM+dg==


In [11]:
symbol = encrypted_result
encoded_symbol = urllib.parse.quote(symbol)
username = urllib.parse.quote(userid)

In [13]:
setcookie=gen_captcha()

CAPTCHA image saved as: captcha_image2.jpg


In [14]:
def check_captcha(csrf, captcha_text, setcookie):
    original_setcookie = setcookie.copy()  # Make a copy of the original setcookie
    url = f"https://www.dgft.gov.in/CP/webHP?requestType=ApplicationRH&actionVal=checkCaptcha&queryType=Select&screenId=114&captcha_val={captcha_text}&_csrf={csrf}"
    response = session.post(url)
    if response.text == '':
        print("Captcha Verified")
        return 1, original_setcookie  # Captcha is verified, return the original setcookie
    else:
        print("Re-enter captcha")
        new_setcookie = gen_captcha()  # Generate new setcookie when captcha verification fails
        return 0, new_setcookie  # Captcha verification failed, return the new setcookie


In [15]:
captcha_text = 'RPY8F'

In [16]:
verification_result, updated_setcookie = check_captcha(csrf, captcha_text, setcookie)


Captcha Verified


In [17]:
updated_setcookie

{'Content-Type': 'image/png', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Mon, 11 Sep 2023 11:20:00 GMT', 'Server': 'Apache', 'X-Frame-Options': 'SAMEORIGIN', 'Cache-Control': 'private,no-cache,no-store', 'Set-Cookie': 'AWSALB=CSogIqEzt06M2+3zMfoEPprF2yRxPgUDeMQ3Lp6m0y9+pZ/sipU9Fz8Izc2ifu/dGPEEgwvomfUXQk3lQuo2+TgnkyE2sHyLJPzCeYN7fqquajYB1yGnqsMv+0az; Expires=Mon, 18 Sep 2023 11:20:00 GMT; Path=/, AWSALBCORS=CSogIqEzt06M2+3zMfoEPprF2yRxPgUDeMQ3Lp6m0y9+pZ/sipU9Fz8Izc2ifu/dGPEEgwvomfUXQk3lQuo2+TgnkyE2sHyLJPzCeYN7fqquajYB1yGnqsMv+0az; Expires=Mon, 18 Sep 2023 11:20:00 GMT; Path=/; SameSite=None, JSESSIONID=F5C29EDF37BDB4F3311AAE81050ED921; Path=/CP; HttpOnly', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 4774b00e2d656478bcba37c033ae541c.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'MAA51-P3', 'Alt-Svc': 'h3=":443"; ma=86400', 'X-Amz-Cf-Id': 'qYt2nwR8rjeOmMOMCJ9S1OVmCiimPK-U-2l0MXDlO1QI_TcviDkxCw=='}

In [18]:
url = "https://www.dgft.gov.in/CP/j_spring_security_check"
data = {
    "username": userid,
    "password": encoded_symbol,
    "txt_Captcha": captcha_text,
    "captcha_val": ""
}
response = session.post(url, data=data)
print("Response status code:", response.status_code)
awsalb_cookie = response.cookies.get('AWSALB')    
jsessionId = response.cookies.get('JSESSIONID')


Response status code: 500


In [19]:
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
if csrf_value := soup.find('meta', attrs={'name': '_csrf'}):
    csrf = csrf_value.get('content')
    print("CSRF Content:", csrf)
    awsalb_cookie = response.cookies.get('AWSALB')
    Jsss = response.cookies.get('JSESSIONID')
else:
    print("CSRF Meta tag not found.")

CSRF Meta tag not found.


In [415]:
url = f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&menuCode=50001&actionVal=loadpage&screenId=50001&appId=201000001&_csrf={csrf}"
data = {
    "portal": "CAS"
}
headers = {
    "Host": "www.dgft.gov.in",
        "Set-Cookie": f"AWSALB={awsalb_cookie}",
        "Set-Cookie": f"JSESSIONID={Jsss}",
         
         "Origin": "https://www.dgft.gov.in",
         }
response = session.post(url, data=data,headers=headers )
if response.status_code == 200:
    print("POST request successful")
    #code to flip the cookie
    awsalb_cookie = response.cookies.get('AWSALB')
    Jsss = response.cookies.get('JSESSIONID')
    print(Jsss)
else:
    print(f"POST request failed with status code: {response.status_code}")
# print(response.text)

POST request successful
BCAB0B16CDE3561FF75F5700FDC9D6D9


In [416]:
import requests
url = f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=isPanExist&screenId=50001&appId=2&_csrf={csrf}"
payload = {
    "panNumber": inp['iecRequestGeneralDetailsDTO']['panNumber']
}
headers = {
    "Host": "www.dgft.gov.in",
        "Set-Cookie": f"AWSALB={awsalb_cookie}",
        "Set-Cookie": f"JSESSIONID={Jsss}",
         
         "Origin": "https://www.dgft.gov.in",
         }
response = session.post(url, data=payload, headers=headers)
if response.status_code == 200:
    print("POST request successful")
    
    awsalb_cookie = response.cookies.get('AWSALB')
    Jsss = response.cookies.get('JSESSIONID')
    
    print(response.text)
else:
    print(f"POST request failed with status code: {response.status_code}")


POST request successful
{"date":null,"iecStatusVal":null,"city":null,"preferredActivity":null,"mrgFileprogressCheck":null,"dateOfIssuance":null,"directorDetails":null,"iecMstId":null,"modificationCheck":null,"natureOfConcern":null,"s3Path":null,"fileNo":null,"isPanExist":0.0,"appId":null,"state":null,"place":null,"pan":null,"cibicStatusVal":null,"iecNo":null,"orderNo":null,"address2":null,"address1":null,"mrgMstcheck":null,"iecNumericStatus":false,"panNumber":null,"cibicStatus":null,"panInNList":false,"district":null,"pinCode":null,"name":null,"iecStatus":null,"lstDate":null,"strDate":null,"branchDetails":null,"crtDt":null,"fileId":null,"status":null}


In [331]:
import time
payload = {
    "panNumber": inp['iecRequestGeneralDetailsDTO']['panNumber'],
    "nameAsPan": inp['iecRequestGeneralDetailsDTO']['nameAsPan'],
    "birthdate": inp['iecRequestGeneralDetailsDTO']['birthdate'],
    "panChange": "true"
}
headers = {
    
    "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
    "Host": "www.dgft.gov.in",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.dgft.gov.in",
    "Referer": "https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=checkLogin",
    }
url = f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=validatePan&screenId=50001&_csrf={csrf}"
while True: 
    response = session.post(url,headers = headers,  data=payload)
    if response.status_code == 200:
        print("POST request successful")
        awsalb_cookie = response.cookies.get('AWSALB')     
        Jsss = response.cookies.get('JSESSIONID')
        print(response.text)
        
        if "PAN Validated Successfully" in response.text:
            print("PAN Validated Successfully. Exiting the loop.")
            break  # Exit the loop if the desired text is present in the response text
    else:
        print(f"POST request failed with status code: {response.status_code}")
    
    time.sleep(2)  # Wait for 2 seconds before sending the next request



POST request successful
{"msgStatus":"true","msgString":"PAN Validated Successfully."}
PAN Validated Successfully. Exiting the loop.


In [417]:
url = f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=getCINData&screenId=50001&_csrf={csrf}"
payload = {
    "iecModule[iecRequestGeneralDetailsDTO][cin]": "undefined",
    "cin": inp['iecRequestGeneralDetailsDTO']['cin'],
    "gstin": ""
}
headers = {
    "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
    "Host": "www.dgft.gov.in",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.dgft.gov.in",
    "Referer": "https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=checkLogin"
}
while True: 
    response = session.post(url,headers=headers, data=payload)
    if response.status_code == 200:
        print("POST request successful")
        awsalb_cookie = response.cookies.get('AWSALB')  
        Jsss = response.cookies.get('JSESSIONID')
        print(response.text)
        json_response = response.json()
        entity_name = json_response.get("entityName")
        if entity_name != "null":
            print(f"Entity Name: {entity_name}")
            print("Exiting the loop.")
            break  # Exit the loop if the entity name is not "null"
    else:
        print(f"POST request failed with status code: {response.status_code}")
    time.sleep(2)  # Wait for 2 seconds before sending the next request


POST request successful
{"iecPanId":null,"panNumber":null,"nameAsPan":null,"birthdate":null,"addressChangedRms":null,"addMobile":null,"addEmail":"aayushi@hattyhood.com","sezEntity":null,"eouEntity":null,"districtLst":null,"stateLst":null,"iecRequestAddressDetailsDTO":null,"iecRequestBranchDetailsDTOs":null,"gstinValidated":null,"cinValidated":null,"preferredActivity":null,"natureOfConcern":null,"json":null,"id":0,"cin":"U82990KA2023PTC172277","gstin":null,"entityName":"BB FINERY PRIVATE LIMITED","entityAddress":null,"addressLine2":"THE PRINCIPAL BMS COLLEGE (ECS ) EDUCATIONAL INSTITUTION , (O&M1),BANASHANKARI, Banashankari III Stage Bangalore South Bangalore KA 560085 IN","addMobileOTP":null,"addEmailOTP":null,"generalDetailsAttachmentDTO":null,"addressDetailsAttachmentDTO":null,"dgftOfficeList":null,"adreessline":null,"city":null,"state":null,"district":null,"pincode":null,"dgft":null,"branchCodes":null,"pleaseSpecifyNatureOfConcern":null,"pleaseSpecifyPreferred":null,"panValidated":n

In [418]:
import requests
from bs4 import BeautifulSoup
# Replace 'YOUR_CSRF_TOKEN_HERE' with your actual CSRF token

url = f"https://www.dgft.gov.in/CP/Upload?attachmentNameHidden=generalAttach&customFunction=&_csrf={csrf}&attachment_type=undefined&_csrf={csrf}"

files = {
    'importFilegeneralAttach': ('206AB PDF.pdf', open(r"Certificate of Incorporation .pdf", 'rb'), 'application/pdf')
}
headers = { 
               
   "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
    }


data = {
    'AttachmentString_generalAttach': '',
    'formId': 'generalForm',
    'customFunction': '',
    'attachmentName': 'generalAttach',
    'attachmentComponentHeader_generalAttach': 'Remarks',
    'maxNoOfAttachmentAllowed_generalAttach': '5',
    'maxFileSizeAllowed_generalAttach': '5',
    'mandatoryFlag_generalAttach': 'Y',
    'allowedExtensions_generalAttach': 'pdf',
    'prevAttachId_generalAttach': 'null',
    'totalFileSize_generalAttach': '0',
    'fileName_generalAttach': '',
}

response = session.post(url, data=data, files=files, headers= headers )

if response.status_code == 200:
    print("POST request successful")
    awsalb_cookie = response.cookies.get('AWSALB') 
    Jsss = response.cookies.get('JSESSIONID')
    # print(response.text)
else:
    print(f"POST request failed with status code: {response.status_code}")

    

# Assuming 'response' contains the response from the server
response_text = response.text

# Parse the HTML content using BeautifulSoup

soup = BeautifulSoup(response_text, 'html.parser')

# Find the input element with id 'AttachmentString_generalAttach'
attachment_input = soup.find('input', {'id': 'AttachmentString_generalAttach'})

# Extract the value attribute
if attachment_input:
    genattach = attachment_input.get('value')
    print("Value of genattach:", genattach)
else:
    print("AttachmentString_generalAttach not found in the response.")

POST request successful
Value of genattach: 2023/09/04/1693829794746_2531071203298724242.tmp* *206AB PDF.pdf*  *0


In [419]:
import requests

url = f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=pincodeData&screenId=99000888&pincode=560004&_csrf={csrf}"#pin code value 
headers = {
    
     "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
     
    }
response =  session.post(url , headers=headers)

if response.status_code == 200:
    print("POST request successful")
    awsalb_cookie = response.cookies.get('AWSALB')   
    Jsss = response.cookies.get('JSESSIONID')
    print(response.text)
else:
    print(f"POST request failed with status code: {response.status_code}")


POST request successful
{"state":{"key":29,"value":"KARNATAKA"},"district":{"key":273,"value":"BENGALURU URBAN"},"dgft":{"key":null,"value":null},"districts":[],"pincode":"560004"}


In [421]:
import requests
url = f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=getDGFTOffice&screenId=50001&_csrf={csrf}"
data = {
    "pincode": inp['iecRequestGeneralDetailsDTO']['pincode'],
    "districtkey": inp['iecRequestGeneralDetailsDTO']['district']['key'],
    "districtvalue": inp['iecRequestGeneralDetailsDTO']['district']['value'],
    "sezEntity": inp['iecRequestGeneralDetailsDTO']['sezEntity']
}

headers = {
               
   "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
  

    
}
response = session.post(url,  data=data , headers=headers)

if response.status_code == 200:
    print("POST request successful")
    awsalb_cookie = response.cookies.get('AWSALB')   
    Jsss = response.cookies.get('JSESSIONID')   
    print(response.text)
else:
    print(f"POST request failed with status code: {response.status_code}")


POST request successful
[{"key":11100008,"value":"RA BENGALURU"}]


In [422]:
url = f"https://www.dgft.gov.in/CP/Upload?attachmentNameHidden=addressAttach&customFunction=&_csrf={csrf}&attachment_type=undefined&_csrf={csrf}"
files = {
    'importFileaddressAttach': ('206AB PDF.pdf', open(r"Certificate of Incorporation .pdf", 'rb'), 'application/pdf')
}
headers = { 
               
   "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
    }
sponse = requests.post(url , files=files , headers = headers)
if response.status_code == 200:
    print("POST request for attaching document successful")
    awsalb_cookie = response.cookies.get('AWSALB') 
    Jsss = response.cookies.get('JSESSIONID')
    # print(response.text)
else:
    print(f"POST request for attaching document failed with status code: {response.status_code}")
response_text = response.text
soup = BeautifulSoup(response_text, 'html.parser')
attachment_input = soup.find('input', {'id': 'AttachmentString_addressAttach'})
if attachment_input:
    Address_attach = attachment_input.get('value')
    print("Value of ADDRESSattach:", Address_attach)
else:
    print("AttachmentString_generalAttach not found in the response.")

POST request for attaching document successful
AttachmentString_generalAttach not found in the response.


In [423]:
import requests

url = f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=pincodeData&screenId=99000888&pincode=560004&_csrf={csrf}"#pin code value 

headers = { 
               
"Set-Cookie": f"AWSALB={awsalb_cookie}",
"Set-Cookie": f"JSESSIONID={Jsss}",
    }

response = session.post(url, headers= headers)

if response.status_code == 200:
    print("POST request successful")
    awsalb_cookie = response.cookies.get('AWSALB')  
    Jsss = response.cookies.get('JSESSIONID')
    print(response.text)
else:
    print(f"POST request failed with status code: {response.status_code}")


POST request successful
{"state":{"key":29,"value":"KARNATAKA"},"district":{"key":273,"value":"BENGALURU URBAN"},"dgft":{"key":null,"value":null},"districts":[],"pincode":"560004"}


In [424]:
inp['iecRequestGeneralDetailsDTO']['sezEntity']

'NO'

In [426]:
import requests
url = f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=getDGFTOffice&screenId=50001&_csrf={csrf}"
data = {
    "pincode": inp['iecRequestGeneralDetailsDTO']['pincode'],
    "districtkey": inp['iecRequestGeneralDetailsDTO']['district']['key'],
    "districtvalue": inp['iecRequestGeneralDetailsDTO']['district']['value'],
    "sezEntity": inp['iecRequestGeneralDetailsDTO']['sezEntity']
}

headers = { 
               
   "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}"
    }

response = session.post(url,  data=data, headers= headers)

if response.status_code == 200:
    print("POST request successful")
    awsalb_cookie = response.cookies.get('AWSALB')  
    Jsss = response.cookies.get('JSESSIONID')
    print(response.text)
else:
    print(f"POST request failed with status code: {response.status_code}")


POST request successful
[{"key":11100008,"value":"RA BENGALURU"}]


In [429]:
#code to flip the cookie
awsalb_cookie = response.cookies.get('AWSALB')     
Jsss = response.cookies.get('JSESSIONID')
#code to flip the cookie
cookie

'JSESSIONID=E00E890AAADFFCCA0006E2D5D8ECADCE;AWSALB=4oQk+dyCNe4VnfPYVfiTn3Uoe1bODZFYG9W4gf8IIapO6rUfKrX0PYDStbr1BLbCaToXMXCGKoD54EYVlCGeIdB8Opny8f0xN49aFrLTD629SNUO6ZTPEbl5Gs/l'

In [430]:
iecModule=r'{"iecRequestDirectorDetailsDTOs":[],"modifyReqType":"Update PAN Details","iecRequestGeneralDetailsDTO":{"deficiency":"","natureOfConcern":{"key":"111000113","value":"Private Limited"},"entityName":"BB FINERY PRIVATE LIMITED","panNumber":"AALCB5105H","nameAsPan":"BB FINERY PRIVATE LIMITED","birthdate":"12/04/2023","preferredActivity":{"key":"111000121","value":"Merchant Exporter"},"sezEntity":"NO","eouEntity":"NO","cin":"U82990KA2023PTC172277","gstin":"","addMobile":"7050541423","mobOrEmailModified":"false","addEmail":"Satvikmishra2615@gmail.com","otpMobile":"","otpEmail":"","pleaseSpecifyNatureOfConcern":"","pleaseSpecifyPreferred":"","AttachmentString_generalAttach":"2023/09/01/1693548532247_2480511743773392403.tmp* *Part B statement (2).pdf* *0","formId":"addForm","customFunction":"","attachmentName":"addressAttach","attachmentComponentHeader_generalAttach":"Remarks","maxNoOfAttachmentAllowed_generalAttach":"5","maxFileSizeAllowed_generalAttach":"5","mandatoryFlag_generalAttach":"Y","allowedExtensions_generalAttach":"pdf","prevAttachId_generalAttach":"","totalFileSize_generalAttach":"0.19","fileName_generalAttach":"Part B statement (2).pdf","entityAddress":"BMS COLLEGE OF ENGINEERING","addressLine2":"BULL TEMPLE ROAD","pincode":"560004","city":"BANGALORE","state":{"key":"29","value":"KARNATAKA"},"district":{"key":"273","value":"BENGALURU URBAN"},"dgft":{"key":"11100008","value":"RA BENGALURU"},"AttachmentString_addressAttach":"2023/09/01/1693548561599_8772271865429898078.tmp* *Part B statement (2).pdf* *0","attachmentComponentHeader_addressAttach":"Remarks","maxNoOfAttachmentAllowed_addressAttach":"5","maxFileSizeAllowed_addressAttach":"5","mandatoryFlag_addressAttach":"Y","allowedExtensions_addressAttach":"pdf","prevAttachId_addressAttach":"","totalFileSize_addressAttach":"0.19","fileName_addressAttach":"Part B statement (2).pdf","sezBranch":"NO","eouBranch":"NO","gstinBranch":"","branchAddressLine1":"","branchAddressLine2":"","branchCity":"","branchPinCode":"","gstinValidated":"","iecRequestBranchDetailsDTOs":[{"sezBranch":"NO","eouBranch":"NO","branchAddressLine1":"ASDA","branchAddressLine2":"ASDA","gstinBranch":"","branchCity":"ASDA","branchDistrict_Value":"BENGALURU URBAN","branchDistrict_key":"273","branchState_Value":"KARNATAKA","branchState_key":"29","branchPinCode":"560004","branchCode":1,"branchStatus":"N","branchPkId":0,"branchManualEntryRms":"false","branchDistrict":{"key":"273","value":"BENGALURU URBAN"},"branchState":{"key":"29","value":"KARNATAKA"},"siNo":1}],"isCINChange":false,"generalDetailsAttachmentDTO":{"s3Path":"2023/09/01/1693548532247_2480511743773392403.tmp* *Part B statement (2).pdf* *0","attachmentType":{"key":"10000000","value":"IEC General Doc"}},"addressDetailsAttachmentDTO":{"s3Path":"2023/09/01/1693548561599_8772271865429898078.tmp* *Part B statement (2).pdf* *0","attachmentType":{"key":"10000001","value":"IEC Address Doc"}}},"reqType":"applyForIEC","deficiencyDto":{"deficiency":"","appId":"201000001","req_file_no":""},"branchIdIncrement":"1","epcgAuthorisationDetailsVoList":[],"aaAuthorisationDetailsVoList":[],"lifecycleDTOs":[],"dfiaAuthorisationVO":[],"panIecNumber":"","mobModified":"","emailModified":""}'


In [431]:
import json

In [432]:
genattach

'2023/09/04/1693829794746_2531071203298724242.tmp* *206AB PDF.pdf*  *0'

In [433]:
iecModule=json.loads(iecModule)

In [434]:
file_path = "data_ie.json"

# Open the file in write mode and save the JSON data
with open(file_path, "w") as json_file:
    json.dump(iecModule, json_file)
    

print("JSON data saved to", file_path)


JSON data saved to data_ie.json


In [436]:
iecModule['iecRequestGeneralDetailsDTO']['AttachmentString_generalAttach']=genattach

In [437]:
iecModule['iecRequestGeneralDetailsDTO']['AttachmentString_addressAttach']=Address_attach

In [438]:
iecModule['iecRequestGeneralDetailsDTO']['generalDetailsAttachmentDTO']['s3Path']=genattach

In [439]:
iecModule['iecRequestGeneralDetailsDTO']['addressDetailsAttachmentDTO']['s3Path']=Address_attach

In [440]:
iecModule

{'iecRequestDirectorDetailsDTOs': [],
 'modifyReqType': 'Update PAN Details',
 'iecRequestGeneralDetailsDTO': {'deficiency': '',
  'natureOfConcern': {'key': '111000113', 'value': 'Private Limited'},
  'entityName': 'BB FINERY PRIVATE LIMITED',
  'panNumber': 'AALCB5105H',
  'nameAsPan': 'BB FINERY PRIVATE LIMITED',
  'birthdate': '12/04/2023',
  'preferredActivity': {'key': '111000121', 'value': 'Merchant Exporter'},
  'sezEntity': 'NO',
  'eouEntity': 'NO',
  'cin': 'U82990KA2023PTC172277',
  'gstin': '',
  'addMobile': '7050541423',
  'mobOrEmailModified': 'false',
  'addEmail': 'Satvikmishra2615@gmail.com',
  'otpMobile': '',
  'otpEmail': '',
  'pleaseSpecifyNatureOfConcern': '',
  'pleaseSpecifyPreferred': '',
  'AttachmentString_generalAttach': '2023/09/04/1693829794746_2531071203298724242.tmp* *206AB PDF.pdf*  *0',
  'formId': 'addForm',
  'customFunction': '',
  'attachmentName': 'addressAttach',
  'attachmentComponentHeader_generalAttach': 'Remarks',
  'maxNoOfAttachmentAllow

In [441]:
i='''{"deficiencyDto":{"deficiency":"","req_file_no":""},"iecRequestAddressDetailsDTO":null,"iecRequestBranchDetailsDTOs":null,"bankDetailsDTOs":[],"iecRequestBankDetailsDTO":null,"iecRequestOtherDetailsDTO":null,"iecRequestDirectorDetailsDTOs":[],"iecRequestGeneralDetailsDTO":{"deficiency":"false","natureOfConcern":{"key":"111000113","value":"Private Limited"},"entityName":"BB FINERY PRIVATE LIMITED","panNumber":"AALCB5105H","nameAsPan":"BB FINERY PRIVATE LIMITED","birthdate":"12/04/2023","preferredActivity":{"key":"111000121","value":"Merchant Exporter"},"sezEntity":"NO","eouEntity":"NO","cin":"U82990KA2023PTC172277","gstin":"","addMobile":"7050541423","mobOrEmailModified":"false","addEmail":"Satvikmishra2615@gmail.com","otpMobile":"","otpEmail":"","pleaseSpecifyNatureOfConcern":"","pleaseSpecifyPreferred":"","AttachmentString_generalAttach0":"2023/09/01/1693558881467_1199406283728543531.tmp* *bipul_pan__4_ (1).pdf*  *0","formId":"addForm","customFunction":"","attachmentName":"addressAttach0","attachmentComponentHeader_generalAttach0":"Remarks","maxNoOfAttachmentAllowed_generalAttach0":"5","maxFileSizeAllowed_generalAttach0":"5","mandatoryFlag_generalAttach0":"Y","allowedExtensions_generalAttach0":"pdf","prevAttachId_generalAttach0":"null","totalFileSize_generalAttach0":"0.00","fileName_generalAttach0":"bipul_pan__4_ (1).pdf","entityAddress":"BMS COLLEGE OF ENGINEERING","addressLine2":"BULL TEMPLE ROAD","pincode":"560004","city":"BANGALORE","state":{"key":"29","value":"KARNATAKA"},"district":{"key":"273","value":"BENGALURU URBAN"},"dgft":{"key":"11100008","value":"RA BENGALURU"},"AttachmentString_addressAttach0":"2023/09/01/1693558947222_6083737094846384320.tmp* *bipul_pan__4_ (1).pdf*  *0","attachmentComponentHeader_addressAttach0":"Remarks","maxNoOfAttachmentAllowed_addressAttach0":"5","maxFileSizeAllowed_addressAttach0":"5","mandatoryFlag_addressAttach0":"Y","allowedExtensions_addressAttach0":"pdf","prevAttachId_addressAttach0":"null","totalFileSize_addressAttach0":"0.00","fileName_addressAttach0":"bipul_pan__4_ (1).pdf","sezBranch":"NO","eouBranch":"NO","gstinBranch":"","branchAddressLine1":"","branchAddressLine2":"","branchCity":"","branchPinCode":"","gstinValidated":"","iecRequestBranchDetailsDTOs":[{"sezBranch":"NO","eouBranch":"NO","branchAddressLine1":"BMSCE","branchAddressLine2":"BULL TEMPLE ROAD","gstinBranch":"","branchCity":"BANGALORE","branchDistrict_Value":"BENGALURU URBAN","branchDistrict_key":"273","branchState_Value":"KARNATAKA","branchState_key":"29","branchPinCode":"560004","branchCode":1,"branchStatus":"N","branchPkId":0,"branchManualEntryRms":"false","branchDistrict":{"key":"273","value":"BENGALURU URBAN"},"branchState":{"key":"29","value":"KARNATAKA"},"siNo":1}],"isCINChange":true,"generalDetailsAttachmentDTO":{"s3Path":"2023/09/01/1693558881467_1199406283728543531.tmp* *bipul_pan__4_ (1).pdf*  *0","attachmentType":{"key":"10000000","value":"IEC General Doc"}},"addressDetailsAttachmentDTO":{"s3Path":"2023/09/01/1693558947222_6083737094846384320.tmp* *bipul_pan__4_ (1).pdf*  *0","attachmentType":{"key":"10000001","value":"IEC Address Doc"}}},"isDigitalSigned":null,"paymentId":null,"reqType":"applyForIEC","address2":null,"isPanExist":null,"dscId":null,"appID":null,"gstin":null,"panDirector":"","delOrders":null,"delStatus":null,"birthdateDir":"","directorNameAsPAN":"","isVisited":true,"checkStatus":null,"checkProfile":null,"fileId":null,"manageIecserviceReq":null,"surrenderflag":null,"revokeSuspensionflag":null,"cin":null,"panNumber":null,"suspendedReasons":null,"pastYearExportDeemedSuppliesDTO":null,"pastYearTableDTO":null,"iecStatusHolderDTO":null,"epcgAuthorisationDetailsVoList":[],"aaAuthorisationDetailsVoList":[],"dfiaAuthorisationVO":[],"rcmcDetailDTOList":null,"industrialDetailDTOList":null,"linkedIECDTOs":null,"lifecycleDTOs":[],"aeoDtlsDTO":null,"insertFlag":null,"updateFlag":null,"deleteFlag":null,"modificationCheck":null,"modificationFileNumber":null,"appId":null,"cibicStatusVal":null,"cibicStatus":null,"declarationCheck":null,"declaration":"","signBy":null,"iecMstId":null,"isIecApplied":null,"iecRole":null,"branchIdIncrement":"1","dirIdIncrement":null,"suspensionDate":null,"cancelDate":null,"iecNo":null,"iecStatusVal":null,"revokeCancellationFlag":false,"authIdIncrement":null,"meisAuthorisationDetailsVoList":null,"rosctlAuthorisationDetailsVoList":null,"gemsearchAuthorisationDetailsVoList":null,"importauthorisationDetailDTOs":null,"scometauthorisationDetailDTOs":null,"exportauthorisationDetailDTOs":null,"searchAuthorisationDetailsVoList":null,"trqsearchAuthorisationDetailsVoList":null,"mobOrEmailModified":null,"mobModified":"","emailModified":"","scripManagementDTOs":null,"exportList":null,"panValidated":null,"place":null,"date":null,"declarationDate":null,"id":0,"iecId":0,"panIecNumber":"AALCB5105H","nameOnPan":null,"entityName":null,"panAsIEC":null,"raOffice":null,"dob":null,"cbdtValidated":null,"natureOfConcern":null,"prefferredActivities":null,"iecStatus":null,"modifyReqType":"Update IEC Details","invokeBy":null,"iecFileNumber":null,"iecFileDate":null,"ecomRef":null,"kyc":null,"kycStatus":null,"iecNumeric":null,"iecNumericStatus":null,"fileNumber":null,"iecNumber":null,"riskFlag":null,"iecCertificateDto":null,"applicationNumber":"","importList":null,"panInNList":false,"manageProfileFlag":null,"processName":null}'''


In [442]:
url = f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=handleRequest&screenId=50001&direction=next&currentTab=generalInformation&fromTab=&fromIndex=&toIndex=&_csrf={csrf}"
headers = {
   
  
    

    "Host": "www.dgft.gov.in",
    "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.dgft.gov.in",
    "Referer": "https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=checkLogin",
    }
data = {
    "iecModule": json.dumps(iecModule),
    "draftId": "",
    "panNumber": inp['iecRequestGeneralDetailsDTO']['panNumber'],
    "nameAsPan": inp['iecRequestGeneralDetailsDTO']['nameAsPan'],
    "birthdate": inp['iecRequestGeneralDetailsDTO']['birthdate'],
}
response = session.post(url, headers=headers, data=data)
awsalb_cookie = response.cookies.get('AWSALB')    
Jsss = response.cookies.get('JSESSIONID')
print(response.status_code)
print(response.text)

200








	
	
	<input type="hidden" id="currentTabJson" value="{&quot;tabName&quot;:&quot;Details of Proprietor/Partner/Director/\tKarta/Managing Trustee&quot;,&quot;viewName&quot;:&quot;detailsOfProperioter&quot;,&quot;moduleName&quot;:&quot;detailsOfProperioter&quot;,&quot;isVisited&quot;:false}" >
	<input type="hidden" id="moduleJson" value="{&quot;deficiencyDto&quot;:{&quot;application_no&quot;:null,&quot;req_file_no&quot;:null,&quot;deficiency_issued_date&quot;:null,&quot;submited_date&quot;:null,&quot;page&quot;:null,&quot;deficiencyUrl&quot;:null,&quot;req_id&quot;:null,&quot;deficiency&quot;:false,&quot;remarks&quot;:null,&quot;attachedBy&quot;:null,&quot;appId&quot;:201000001},&quot;iecRequestAddressDetailsDTO&quot;:null,&quot;iecRequestBranchDetailsDTOs&quot;:null,&quot;bankDetailsDTOs&quot;:[],&quot;iecRequestBankDetailsDTO&quot;:null,&quot;iecRequestOtherDetailsDTO&quot;:null,&quot;iecRequestDirectorDetailsDTOs&quot;:[{&quot;siNo&quot;:null,&quot;din&quot;:&quot;000973065

In [443]:
from bs4 import BeautifulSoup
import requests

response_html = response.text

# Create a BeautifulSoup object
soup = BeautifulSoup(response_html, 'html.parser')

# Extract the value of applicationNumber
application_number_input = soup.find('input', {'id': 'applicationNumber'})
if application_number_input:
    application_number = application_number_input.get('value')
    print("Application Number:", application_number)
else:
    print("Application Number not found in the response.")
    

# Extract the value of draftId
draft_id_input = soup.find('input', {'id': 'draftId'})
if draft_id_input:
    draft_id = draft_id_input.get('value')
    print("Draft ID:", draft_id)
else:
    print("Draft ID not found in the response.")

Application Number: ARNIECPAPPLY02936414AM24
Draft ID: 5568770


In [444]:
prop='''{"deficiencyDto":{"application_no":null,"req_file_no":null,"deficiency_issued_date":null,"submited_date":null,"page":null,"deficiencyUrl":null,"req_id":null,"deficiency":false,"remarks":null,"attachedBy":null,"appId":201000001},"iecRequestAddressDetailsDTO":null,"iecRequestBranchDetailsDTOs":null,"bankDetailsDTOs":[],"iecRequestBankDetailsDTO":null,"iecRequestOtherDetailsDTO":null,"iecRequestDirectorDetailsDTOs":[{"ifForeignDirector":"N","din":"0009730658","title_Value":"Mr.","title_key":"111000128","panDirector":"CIKPK4752M","passportNumber":"","birthdateDir":"01/09/1992","fatherName":"SHYAM BIHARI SINGH","addressLine1":"GRAM PAWARA POST","addressLine2":"","city":"KUSUMHARA","directorName":"PAWAN KUMAR","directorNameAsPAN":"PAWAN KUMAR","directorDistrict_Value":"ROHTAS","directorDistrict_key":"70","directorState_Value":"BIHAR","directorState_key":"10","directorPin":"802218","directorMobile":"7991162910","directorId":"","directorManualEntryRms":"false","mcaValidated":"true","directorDistrict":{"key":"70","value":"ROHTAS"},"directorState":{"key":"10","value":"BIHAR"},"title":{"key":"111000128","value":"Mr."}},{"ifForeignDirector":"N","din":"0010113480","title_Value":"Ms.","title_key":"111000127","panDirector":"NNKPS1275K","passportNumber":"","birthdateDir":"11/10/2000","fatherName":"SUNIL KUMAR SINGH","addressLine1":"GITA APARTMENTS HOUSE NO 32","addressLine2":"","city":"PATLIPUTRA","directorName":"AAYUSHI SINGH","directorNameAsPAN":"AAYUSHI SINGH","directorDistrict_Value":"PATNA","directorDistrict_key":"73","directorState_Value":"BIHAR","directorState_key":"10","directorPin":"800013","directorMobile":"8102014565","directorId":"","directorManualEntryRms":"false","mcaValidated":"true","directorDistrict":{"key":"73","value":"PATNA"},"directorState":{"key":"10","value":"BIHAR"},"title":{"key":"111000127","value":"Ms."}}],"iecRequestGeneralDetailsDTO":{"iecPanId":null,"panNumber":"AALCB5105H","nameAsPan":"BB FINERY PRIVATE LIMITED","birthdate":"12/04/2023","addressChangedRms":null,"addMobile":"7050541423","addEmail":"Satvikmishra2615@gmail.com","sezEntity":"NO","eouEntity":"NO","districtLst":null,"stateLst":null,"iecRequestAddressDetailsDTO":null,"iecRequestBranchDetailsDTOs":[],"gstinValidated":"","cinValidated":null,"preferredActivity":{"key":111000121,"value":"Merchant Exporter"},"natureOfConcern":{"key":111000113,"value":"Private Limited"},"json":null,"id":0,"cin":"U82990KA2023PTC172277","gstin":"","entityName":"BB FINERY PRIVATE LIMITED","entityAddress":"THE PRINCIPAL BMS COLLEGE (ECS)","addressLine2":"EDUCATIONAL INSTITUTION, (0&M1), BANASHANKARI,","addMobileOTP":null,"addEmailOTP":null,"generalDetailsAttachmentDTO":{"attachId":null,"s3Path":"2023/09/01/1693564252867_1486129455494450877.tmp* *Certificate of Incorporation .pdf*  *0","fileName":null,"fileType":null,"fileSize":null,"attachmentType":{"key":10000000,"value":"IEC General Doc"},"remark":null,"refTableId":null,"pkId":null,"hashDocuments":[{"path":"2023/09/01/1693564252867_1486129455494450877.tmp","name":"Certificate of Incorporation .pdf","hash":"e59dd1f23152d3d5b0b7a0f6ea107bfc5f293824c5c09868f845e1f52b2cdc78"}],"attachDate":null,"update":false},"addressDetailsAttachmentDTO":{"attachId":null,"s3Path":"2023/09/01/1693564437118_5603255190324649268.tmp* *Electricity.pdf*  *0","fileName":null,"fileType":null,"fileSize":null,"attachmentType":{"key":10000001,"value":"IEC Address Doc"},"remark":null,"refTableId":null,"pkId":null,"hashDocuments":[{"path":"2023/09/01/1693564437118_5603255190324649268.tmp","name":"Electricity.pdf","hash":"c574de59f1292944add9c1a42b6795e3b4026a0bcd5f895bd4250fb595555d6f"}],"attachDate":null,"update":false},"dgftOfficeList":null,"adreessline":null,"city":"Bengaluru","state":{"key":29,"value":"KARNATAKA"},"district":{"key":273,"value":"BENGALURU URBAN"},"pincode":"560085","dgft":{"key":11100008,"value":"RA BENGALURU"},"branchCodes":null,"pleaseSpecifyNatureOfConcern":"","pleaseSpecifyPreferred":"","panValidated":null,"isCINChange":true},"isDigitalSigned":null,"paymentId":null,"reqType":"applyForIEC","address2":null,"isPanExist":null,"dscId":null,"appID":null,"gstin":null,"panDirector":"","delOrders":null,"delStatus":null,"birthdateDir":"","directorNameAsPAN":"","isVisited":true,"checkStatus":null,"checkProfile":null,"fileId":null,"manageIecserviceReq":null,"surrenderflag":null,"revokeSuspensionflag":null,"cin":null,"panNumber":null,"suspendedReasons":null,"pastYearExportDeemedSuppliesDTO":null,"pastYearTableDTO":null,"iecStatusHolderDTO":null,"epcgAuthorisationDetailsVoList":[],"aaAuthorisationDetailsVoList":[],"dfiaAuthorisationVO":[],"rcmcDetailDTOList":null,"industrialDetailDTOList":null,"linkedIECDTOs":null,"lifecycleDTOs":[],"aeoDtlsDTO":null,"insertFlag":null,"updateFlag":null,"deleteFlag":null,"modificationCheck":null,"modificationFileNumber":null,"appId":null,"cibicStatusVal":null,"cibicStatus":null,"declarationCheck":null,"declaration":"","signBy":null,"iecMstId":null,"isIecApplied":null,"iecRole":null,"branchIdIncrement":null,"dirIdIncrement":"","suspensionDate":null,"cancelDate":null,"iecNo":null,"iecStatusVal":null,"revokeCancellationFlag":false,"authIdIncrement":null,"meisAuthorisationDetailsVoList":null,"rosctlAuthorisationDetailsVoList":null,"gemsearchAuthorisationDetailsVoList":null,"importauthorisationDetailDTOs":null,"scometauthorisationDetailDTOs":null,"exportauthorisationDetailDTOs":null,"searchAuthorisationDetailsVoList":null,"trqsearchAuthorisationDetailsVoList":null,"mobOrEmailModified":null,"mobModified":null,"emailModified":null,"scripManagementDTOs":null,"exportList":null,"panValidated":null,"place":null,"date":null,"declarationDate":null,"id":0,"iecId":0,"panIecNumber":"AALCB5105H","nameOnPan":null,"entityName":null,"panAsIEC":null,"raOffice":null,"dob":null,"cbdtValidated":null,"natureOfConcern":null,"prefferredActivities":null,"iecStatus":null,"modifyReqType":"Update PAN Details","invokeBy":null,"iecFileNumber":null,"iecFileDate":null,"ecomRef":null,"kyc":null,"kycStatus":null,"iecNumeric":null,"iecNumericStatus":null,"fileNumber":null,"iecNumber":null,"riskFlag":null,"iecCertificateDto":null,"applicationNumber":"ARNIECPAPPLY02889471AM24","importList":null,"panInNList":false,"manageProfileFlag":null,"processName":null,"ifForeignDirector":"N"}'''
prop=json.loads(prop)
prop['applicationNumber']=  application_number




In [445]:
prop

{'deficiencyDto': {'application_no': None,
  'req_file_no': None,
  'deficiency_issued_date': None,
  'submited_date': None,
  'page': None,
  'deficiencyUrl': None,
  'req_id': None,
  'deficiency': False,
  'remarks': None,
  'attachedBy': None,
  'appId': 201000001},
 'iecRequestAddressDetailsDTO': None,
 'iecRequestBranchDetailsDTOs': None,
 'bankDetailsDTOs': [],
 'iecRequestBankDetailsDTO': None,
 'iecRequestOtherDetailsDTO': None,
 'iecRequestDirectorDetailsDTOs': [{'ifForeignDirector': 'N',
   'din': '0009730658',
   'title_Value': 'Mr.',
   'title_key': '111000128',
   'panDirector': 'CIKPK4752M',
   'passportNumber': '',
   'birthdateDir': '01/09/1992',
   'fatherName': 'SHYAM BIHARI SINGH',
   'addressLine1': 'GRAM PAWARA POST',
   'addressLine2': '',
   'city': 'KUSUMHARA',
   'directorName': 'PAWAN KUMAR',
   'directorNameAsPAN': 'PAWAN KUMAR',
   'directorDistrict_Value': 'ROHTAS',
   'directorDistrict_key': '70',
   'directorState_Value': 'BIHAR',
   'directorState_key':

In [446]:
#details of propietor






url = f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=handleRequest&screenId=50001&direction=next&currentTab=detailsOfProperioter&fromTab=&fromIndex=&toIndex=&_csrf={csrf}"


headers = {
    "Host": "www.dgft.gov.in",
    "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.dgft.gov.in",
    "Referer": "https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=checkLogin",
    }


data = {
    "iecModule": json.dumps(prop),
    "draftId": f"{draft_id}",
    "din": "",
    "panDirector": "",
    "birthdateDir": "",
}
response = session.post(url, headers=headers, data=data)
awsalb_cookie = response.cookies.get('AWSALB')   
Jsss = response.cookies.get('JSESSIONID')
print(response.status_code)
print(response.text)

200









<style>
.dis_validate {
	display: block !important;
	color: red !important;
}

.noDisplay {
	display: none !important;
}

.error-msg {
	color: #f10909 !important;
	font-size: 10px;
	font-weight: bold;
}

.error-msg-text {
	color: #f10909 !important;
	font-size: 10px;
	font-weight: bold;
}

.warning-chk {
	color: #f10909 !important;
	font-size: 10px;
	font-weight: bold;
}
</style>



	
	
	<input type="hidden" id="currentTabJson" value="{&quot;tabName&quot;:&quot;Bank Information&quot;,&quot;viewName&quot;:&quot;bankInformation&quot;,&quot;moduleName&quot;:&quot;bankInformation&quot;,&quot;isVisited&quot;:false}" >
	<input type="hidden" id="moduleJson" value="{&quot;deficiencyDto&quot;:{&quot;application_no&quot;:null,&quot;req_file_no&quot;:null,&quot;deficiency_issued_date&quot;:null,&quot;submited_date&quot;:null,&quot;page&quot;:null,&quot;deficiencyUrl&quot;:null,&quot;req_id&quot;:null,&quot;deficiency&quot;:false,&quot;remarks&quot;:null,&quot;attachedBy&quot;:null,&q

In [447]:
url = f"https://www.dgft.gov.in/CP/Upload?attachmentNameHidden=1&attachmentNameSingle=attach_bank&attachmentFormName=bankForm&_csrf={csrf}&reDirectPage=singleDgftAttachment&dgftSingleAllowedExtensions=pdf&_csrf={csrf}"

files = {
    'importFileaddressAttach': ('206AB PDF.pdf', open(r"cancelled cheque.pdf", 'rb'), 'application/pdf')
}
headers = { 
               
   "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}"
    }

response = session.post(url , files=files, headers= headers)
if response.status_code == 200:
    print("POST request for attaching document successful")
    awsalb_cookie = response.cookies.get('AWSALB')  
    Jsss = response.cookies.get('JSESSIONID')
    # print(response.text)
else:
    print(f"POST request for attaching document failed with status code: {response.status_code}")
response_text = response.text

soup = BeautifulSoup(response_text, "html.parser")
aswS3Path_input = soup.find("input", {"name": "aswS3Path"})

if aswS3Path_input:
    aswS3Path_value = aswS3Path_input.get("value")
    print(aswS3Path_value)
else:
    print("Input with name 'aswS3Path' not found in the response.")
print(response.text)

POST request for attaching document successful
0* *











	
		
		
		
		
		
		
	
	


<style>
.history-document th {
	background-color: #03A9F4;
	color: #fff;
	text-align: center;
	font-family: Open sans;
	font-weight: 400;
	font-size: 10px;
}

div.browse-wrap {
	padding: 10px 0px;
	text-align: center;
	position: relative;
	background-color: #fff;
	border: solid 3px #d2d2d7;
	border-style: dashed;
	top: 0;
	left: 0;
	cursor: pointer;
}

div.title {
	color: #3b5998;
	font-size: 14px;
	font-weight: bold;
	font-family: tahoma, arial, sans-serif;
}

input.upload {
	opacity: 0;
	position: absolute;
	font-size: 12px;
	right: 0;
	margin: 0;
	bottom: 0;
	top: 0;
	left: 0;
	width:100%;
	cursor: inherit;
}
</style>
<div class="row">

	<div class="col-md-12">

		
		
		
			
		
		 
		 
			
		
			<div class="forDelete display-on-validate">Add Attachment</div>

		<a class="title" href="javascript:void(0);"
			onclick="javascript:viewAttachment_single_attach_bank('0* *')">
		</a>
		
		
		<div class

In [448]:
bank = '''{"deficiencyDto":{"application_no":null,"req_file_no":null,"deficiency_issued_date":null,"submited_date":null,"page":null,"deficiencyUrl":null,"req_id":null,"deficiency":false,"remarks":null,"attachedBy":null,"appId":201000001},"iecRequestAddressDetailsDTO":null,"iecRequestBranchDetailsDTOs":null,"bankDetailsDTOs":[{"siNo":"1","accNumber":"10107749364","accHolder":"BB FINERY","ifsCode":"IDFB0080176","bankNameKeyValueDto_Value":"IDFC BANK","bankNameKeyValueDto_key":"0","branchAddress":"Cunningham","isPrimary":"YES","pfmsStatus":"Not Initiated","bankAttachmentDTO":{"attachId":null,"s3Path":"2023/09/01/1693567420823_2960784951405509880.tmp* *cancelled cheque.pdf","fileName":"cancelled cheque.pdf","fileType":null,"fileSize":null,"attachmentType":{"key":10000002,"value":"IEC Bank DOC"},"remark":null,"refTableId":null,"pkId":null,"hashDocuments":[{"path":"2023/09/01/1693567420823_2960784951405509880.tmp","name":"cancelled cheque.pdf","hash":"e6c11f876295ce3a7efb2324669d24f399024e77d9a66b814cd85d7ecbe47f5f"}],"attachDate":null,"update":false},"pfmsValidated":"0","bankPkId":"0","bankNameKeyValueDto":{"key":"0","value":"IDFC BANK"},"bankName":"IDFC BANK"}],"iecRequestBankDetailsDTO":{"accNumber":"","accHolder":"","ifsCode":"","bankNameKeyValueDto":{"key":"-1","value":"Please Select"},"isMandatory":"1","bankName":"IDFC BANK","branchAddress":"","aswS3Path":"","fileName":"","attachmentType":"101","attachmentStr":"","dgftSingleAllowedExtensions":"pdf","maxFileSize":"","maxFileSizeMsg":"","bankValidRms":"false","bankAttachmentDTO":{}},"iecRequestOtherDetailsDTO":null,"iecRequestDirectorDetailsDTOs":[{"siNo":1,"din":"0009730658","dinInt":"9730658","ifForeignDirector":"N","passportNumber":"","directorId":null,"iecDirectorId":null,"panDirector":"CIKPK4752M","fatherName":"SHYAM BIHARI SINGH","birthdateDir":"01/09/1992","addressLine1":"GRAM PAWARA POST","addressLine2":"","city":"KUSUMHARA","directorState_key":10,"title_key":111000128,"title_Value":"Mr.","directorState_Value":"BIHAR","directorDistrict_Value":"ROHTAS","directorDistrict_key":70,"mcaValidated":"true","title":{"key":111000128,"value":"Mr."},"directorName":"PAWAN KUMAR","directorNameAsPAN":"PAWAN KUMAR","directorDistrict":{"key":70,"value":"ROHTAS"},"directorState":{"key":10,"value":"BIHAR"},"directorPin":"802218","directorMobile":"","directorDistrictLst":null,"directorStateLst":null},{"siNo":2,"din":"0010113480","dinInt":"10113480","ifForeignDirector":"N","passportNumber":"","directorId":null,"iecDirectorId":null,"panDirector":"NNKPS1275K","fatherName":"SUNIL KUMAR SINGH","birthdateDir":"11/10/2000","addressLine1":"GITA APARTMENTS HOUSE NO 32","addressLine2":"","city":"PATLIPUTRA","directorState_key":10,"title_key":111000127,"title_Value":"Ms.","directorState_Value":"BIHAR","directorDistrict_Value":"PATNA","directorDistrict_key":73,"mcaValidated":"true","title":{"key":111000127,"value":"Ms."},"directorName":"AAYUSHI SINGH","directorNameAsPAN":"AAYUSHI SINGH","directorDistrict":{"key":73,"value":"PATNA"},"directorState":{"key":10,"value":"BIHAR"},"directorPin":"800013","directorMobile":"","directorDistrictLst":null,"directorStateLst":null}],"iecRequestGeneralDetailsDTO":{"iecPanId":null,"panNumber":"AALCB5105H","nameAsPan":"BB FINERY PRIVATE LIMITED","birthdate":"12/04/2023","addressChangedRms":null,"addMobile":"7050541423","addEmail":"Satvikmishra2615@gmail.com","sezEntity":"NO","eouEntity":"NO","districtLst":null,"stateLst":null,"iecRequestAddressDetailsDTO":null,"iecRequestBranchDetailsDTOs":[],"gstinValidated":"","cinValidated":null,"preferredActivity":{"key":111000121,"value":"Merchant Exporter"},"natureOfConcern":{"key":111000113,"value":"Private Limited"},"json":null,"id":0,"cin":"U82990KA2023PTC172277","gstin":"","entityName":"BB FINERY PRIVATE LIMITED","entityAddress":"THE PRINCIPAL BMS COLLEGE (ECS)","addressLine2":"EDUCATIONAL INSTITUTION, (0&M1), BANASHANKARI,","addMobileOTP":null,"addEmailOTP":null,"generalDetailsAttachmentDTO":{"attachId":null,"s3Path":"2023/09/01/1693564252867_1486129455494450877.tmp* *Certificate of Incorporation .pdf*  *0","fileName":null,"fileType":null,"fileSize":null,"attachmentType":{"key":10000000,"value":"IEC General Doc"},"remark":null,"refTableId":null,"pkId":null,"hashDocuments":[{"path":"2023/09/01/1693564252867_1486129455494450877.tmp","name":"Certificate of Incorporation .pdf","hash":"e59dd1f23152d3d5b0b7a0f6ea107bfc5f293824c5c09868f845e1f52b2cdc78"}],"attachDate":null,"update":false},"addressDetailsAttachmentDTO":{"attachId":null,"s3Path":"2023/09/01/1693564437118_5603255190324649268.tmp* *Electricity.pdf*  *0","fileName":null,"fileType":null,"fileSize":null,"attachmentType":{"key":10000001,"value":"IEC Address Doc"},"remark":null,"refTableId":null,"pkId":null,"hashDocuments":[{"path":"2023/09/01/1693564437118_5603255190324649268.tmp","name":"Electricity.pdf","hash":"c574de59f1292944add9c1a42b6795e3b4026a0bcd5f895bd4250fb595555d6f"}],"attachDate":null,"update":false},"dgftOfficeList":null,"adreessline":null,"city":"Bengaluru","state":{"key":29,"value":"KARNATAKA"},"district":{"key":273,"value":"BENGALURU URBAN"},"pincode":"560085","dgft":{"key":11100008,"value":"RA BENGALURU"},"branchCodes":null,"pleaseSpecifyNatureOfConcern":"","pleaseSpecifyPreferred":"","panValidated":null,"isCINChange":true},"isDigitalSigned":null,"paymentId":null,"reqType":"applyForIEC","address2":null,"isPanExist":null,"dscId":null,"appID":null,"gstin":null,"panDirector":"","delOrders":null,"delStatus":null,"birthdateDir":"","directorNameAsPAN":"","isVisited":true,"checkStatus":null,"checkProfile":null,"fileId":null,"manageIecserviceReq":null,"surrenderflag":null,"revokeSuspensionflag":null,"cin":null,"panNumber":null,"suspendedReasons":null,"pastYearExportDeemedSuppliesDTO":null,"pastYearTableDTO":null,"iecStatusHolderDTO":null,"epcgAuthorisationDetailsVoList":[],"aaAuthorisationDetailsVoList":[],"dfiaAuthorisationVO":[],"rcmcDetailDTOList":null,"industrialDetailDTOList":null,"linkedIECDTOs":null,"lifecycleDTOs":[],"aeoDtlsDTO":null,"insertFlag":null,"updateFlag":null,"deleteFlag":null,"modificationCheck":null,"modificationFileNumber":null,"appId":null,"cibicStatusVal":null,"cibicStatus":null,"declarationCheck":null,"declaration":"","signBy":null,"iecMstId":null,"isIecApplied":null,"iecRole":null,"branchIdIncrement":null,"dirIdIncrement":null,"suspensionDate":null,"cancelDate":null,"iecNo":null,"iecStatusVal":null,"revokeCancellationFlag":false,"authIdIncrement":null,"meisAuthorisationDetailsVoList":null,"rosctlAuthorisationDetailsVoList":null,"gemsearchAuthorisationDetailsVoList":null,"importauthorisationDetailDTOs":null,"scometauthorisationDetailDTOs":null,"exportauthorisationDetailDTOs":null,"searchAuthorisationDetailsVoList":null,"trqsearchAuthorisationDetailsVoList":null,"mobOrEmailModified":null,"mobModified":null,"emailModified":null,"scripManagementDTOs":null,"exportList":null,"panValidated":null,"place":null,"date":null,"declarationDate":null,"id":0,"iecId":0,"panIecNumber":"AALCB5105H","nameOnPan":null,"entityName":null,"panAsIEC":null,"raOffice":null,"dob":null,"cbdtValidated":null,"natureOfConcern":null,"prefferredActivities":null,"iecStatus":null,"modifyReqType":"Update IEC Details","invokeBy":null,"iecFileNumber":null,"iecFileDate":null,"ecomRef":null,"kyc":null,"kycStatus":null,"iecNumeric":null,"iecNumericStatus":null,"fileNumber":null,"iecNumber":null,"riskFlag":null,"iecCertificateDto":null,"applicationNumber":"ARNIECPAPPLY02889471AM24","importList":null,"panInNList":false,"manageProfileFlag":null,"processName":null}'''
bank = json.loads(bank)
bank['applicationNumber']=  application_number


In [449]:
bank['bankDetailsDTOs'][0]['bankNameKeyValueDto_Value']


'IDFC BANK'

In [450]:
url = f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=handleRequest&screenId=50001&direction=current&currentTab=otherDetails&fromTab=bankInformation&fromIndex=2&toIndex=3&_csrf={csrf}"


headers = {
    "Host": "www.dgft.gov.in",
    "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.dgft.gov.in",
    "Referer": "https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=checkLogin",
    }


data = {
    "iecModule": json.dumps(bank),
    "bankName": bank['bankDetailsDTOs'][0]['bankNameKeyValueDto_Value'],
    "branchAddress": "",
    "accHolder": "",
    "birthdateDir": "",
}
response = session.post(url, headers=headers, data=data)
awsalb_cookie = response.cookies.get('AWSALB')   
Jsss = response.cookies.get('JSESSIONID')
print(response.status_code)
print(response.text)




200






	
	
	<input type="hidden" id="currentTabJson" value="{&quot;tabName&quot;:&quot;Other Details (Preferred sectors of operations)&quot;,&quot;viewName&quot;:&quot;otherDetails&quot;,&quot;moduleName&quot;:&quot;otherDetails&quot;,&quot;isVisited&quot;:false}" >
	<input type="hidden" id="moduleJson" value="{&quot;deficiencyDto&quot;:{&quot;application_no&quot;:null,&quot;req_file_no&quot;:null,&quot;deficiency_issued_date&quot;:null,&quot;submited_date&quot;:null,&quot;page&quot;:null,&quot;deficiencyUrl&quot;:null,&quot;req_id&quot;:null,&quot;deficiency&quot;:false,&quot;remarks&quot;:null,&quot;attachedBy&quot;:null,&quot;appId&quot;:201000001},&quot;iecRequestAddressDetailsDTO&quot;:null,&quot;iecRequestBranchDetailsDTOs&quot;:null,&quot;bankDetailsDTOs&quot;:[{&quot;faxnumber&quot;:null,&quot;isNewBankDetails&quot;:null,&quot;isMandatory&quot;:null,&quot;bankNameKeyValueDto&quot;:{&quot;key&quot;:0,&quot;value&quot;:&quot;IDFC BANK&quot;},&quot;bankName&quot;:&quot;IDFC BAN

In [451]:
det = '''{"deficiencyDto":{"application_no":null,"req_file_no":null,"deficiency_issued_date":null,"submited_date":null,"page":null,"deficiencyUrl":null,"req_id":null,"deficiency":false,"remarks":null,"attachedBy":null,"appId":201000001},"iecRequestAddressDetailsDTO":null,"iecRequestBranchDetailsDTOs":null,"bankDetailsDTOs":[{"faxnumber":null,"isNewBankDetails":null,"isMandatory":null,"bankNameKeyValueDto":{"key":0,"value":"IDFC BANK"},"bankName":"IDFC BANK","branchAddress":"Cunningham","accNumber":"10107749364","accHolder":"BB FINERY","ifsCode":"IDFB0080176","json":null,"bankFlag":null,"bankValidRms":null,"bankAttachmentDTO":{"attachId":null,"s3Path":"2023/09/01/1693567420823_2960784951405509880.tmp* *cancelled cheque.pdf","fileName":"cancelled cheque.pdf","fileType":null,"fileSize":null,"attachmentType":{"key":10000002,"value":"IEC Bank DOC"},"remark":null,"refTableId":null,"pkId":null,"hashDocuments":[{"path":"2023/09/01/1693567420823_2960784951405509880.tmp","name":"cancelled cheque.pdf","hash":"e6c11f876295ce3a7efb2324669d24f399024e77d9a66b814cd85d7ecbe47f5f"}],"attachDate":null,"update":false},"applicantName":null,"applicantPanNo":null,"accType":null,"pfmsValidated":0,"isPrimary":"YES","pfmsStatus":"Not Initiated","bankPkId":0,"dobApplicant":null,"aplicantAddress":null,"applicantAddress1":null,"applicantMobile":null,"applicantEmail":null,"applicantAddress2":null}],"iecRequestBankDetailsDTO":{"faxnumber":null,"isNewBankDetails":null,"isMandatory":1,"bankNameKeyValueDto":{"key":-1,"value":"Please Select"},"bankName":"IDFC BANK","branchAddress":"","accNumber":"","accHolder":"","ifsCode":"","json":null,"bankFlag":null,"bankValidRms":false,"bankAttachmentDTO":{"attachId":null,"s3Path":null,"fileName":null,"fileType":null,"fileSize":null,"attachmentType":null,"remark":null,"refTableId":null,"pkId":null,"hashDocuments":null,"attachDate":null,"update":false},"applicantName":null,"applicantPanNo":null,"accType":null,"pfmsValidated":null,"isPrimary":null,"pfmsStatus":null,"bankPkId":null,"dobApplicant":null,"aplicantAddress":null,"applicantAddress1":null,"applicantMobile":null,"applicantEmail":null,"applicantAddress2":null},"iecRequestOtherDetailsDTO":{"importList":[],"importServicesList":[],"exportList":[],"exportServicesList":[],"exportServices":{"key":"0000000","value":"-"},"exportGoodsOtherSpecify":"","exportGoodsOther":false,"importGoodsOtherSpecify":"","importGoodsOther":false,"please_specify":"","isImport":false,"isExport":true,"isOther":false,"isGoods":false,"isServices":false,"importGoods":false,"importServices":false},"iecRequestDirectorDetailsDTOs":[{"siNo":1,"din":"0009730658","dinInt":"9730658","ifForeignDirector":"N","passportNumber":"","directorId":null,"iecDirectorId":null,"panDirector":"CIKPK4752M","fatherName":"SHYAM BIHARI SINGH","birthdateDir":"01/09/1992","addressLine1":"GRAM PAWARA POST","addressLine2":"","city":"KUSUMHARA","directorState_key":10,"title_key":111000128,"title_Value":"Mr.","directorState_Value":"BIHAR","directorDistrict_Value":"ROHTAS","directorDistrict_key":70,"mcaValidated":"true","title":{"key":111000128,"value":"Mr."},"directorName":"PAWAN KUMAR","directorNameAsPAN":"PAWAN KUMAR","directorDistrict":{"key":70,"value":"ROHTAS"},"directorState":{"key":10,"value":"BIHAR"},"directorPin":"802218","directorMobile":"","directorDistrictLst":null,"directorStateLst":null},{"siNo":2,"din":"0010113480","dinInt":"10113480","ifForeignDirector":"N","passportNumber":"","directorId":null,"iecDirectorId":null,"panDirector":"NNKPS1275K","fatherName":"SUNIL KUMAR SINGH","birthdateDir":"11/10/2000","addressLine1":"GITA APARTMENTS HOUSE NO 32","addressLine2":"","city":"PATLIPUTRA","directorState_key":10,"title_key":111000127,"title_Value":"Ms.","directorState_Value":"BIHAR","directorDistrict_Value":"PATNA","directorDistrict_key":73,"mcaValidated":"true","title":{"key":111000127,"value":"Ms."},"directorName":"AAYUSHI SINGH","directorNameAsPAN":"AAYUSHI SINGH","directorDistrict":{"key":73,"value":"PATNA"},"directorState":{"key":10,"value":"BIHAR"},"directorPin":"800013","directorMobile":"","directorDistrictLst":null,"directorStateLst":null}],"iecRequestGeneralDetailsDTO":{"iecPanId":null,"panNumber":"AALCB5105H","nameAsPan":"BB FINERY PRIVATE LIMITED","birthdate":"12/04/2023","addressChangedRms":null,"addMobile":"7050541423","addEmail":"Satvikmishra2615@gmail.com","sezEntity":"NO","eouEntity":"NO","districtLst":null,"stateLst":null,"iecRequestAddressDetailsDTO":null,"iecRequestBranchDetailsDTOs":[],"gstinValidated":"","cinValidated":null,"preferredActivity":{"key":111000121,"value":"Merchant Exporter"},"natureOfConcern":{"key":111000113,"value":"Private Limited"},"json":null,"id":0,"cin":"U82990KA2023PTC172277","gstin":"","entityName":"BB FINERY PRIVATE LIMITED","entityAddress":"THE PRINCIPAL BMS COLLEGE (ECS)","addressLine2":"EDUCATIONAL INSTITUTION, (0&M1), BANASHANKARI,","addMobileOTP":null,"addEmailOTP":null,"generalDetailsAttachmentDTO":{"attachId":null,"s3Path":"2023/09/01/1693564252867_1486129455494450877.tmp* *Certificate of Incorporation .pdf*  *0","fileName":null,"fileType":null,"fileSize":null,"attachmentType":{"key":10000000,"value":"IEC General Doc"},"remark":null,"refTableId":null,"pkId":null,"hashDocuments":[{"path":"2023/09/01/1693564252867_1486129455494450877.tmp","name":"Certificate of Incorporation .pdf","hash":"e59dd1f23152d3d5b0b7a0f6ea107bfc5f293824c5c09868f845e1f52b2cdc78"}],"attachDate":null,"update":false},"addressDetailsAttachmentDTO":{"attachId":null,"s3Path":"2023/09/01/1693564437118_5603255190324649268.tmp* *Electricity.pdf*  *0","fileName":null,"fileType":null,"fileSize":null,"attachmentType":{"key":10000001,"value":"IEC Address Doc"},"remark":null,"refTableId":null,"pkId":null,"hashDocuments":[{"path":"2023/09/01/1693564437118_5603255190324649268.tmp","name":"Electricity.pdf","hash":"c574de59f1292944add9c1a42b6795e3b4026a0bcd5f895bd4250fb595555d6f"}],"attachDate":null,"update":false},"dgftOfficeList":null,"adreessline":null,"city":"Bengaluru","state":{"key":29,"value":"KARNATAKA"},"district":{"key":273,"value":"BENGALURU URBAN"},"pincode":"560085","dgft":{"key":11100008,"value":"RA BENGALURU"},"branchCodes":null,"pleaseSpecifyNatureOfConcern":"","pleaseSpecifyPreferred":"","panValidated":null,"isCINChange":true},"isDigitalSigned":null,"paymentId":null,"reqType":"applyForIEC","address2":null,"isPanExist":null,"dscId":null,"appID":null,"gstin":null,"panDirector":"","delOrders":null,"delStatus":null,"birthdateDir":"","directorNameAsPAN":"","isVisited":true,"checkStatus":null,"checkProfile":null,"fileId":null,"manageIecserviceReq":null,"surrenderflag":null,"revokeSuspensionflag":null,"cin":null,"panNumber":null,"suspendedReasons":null,"pastYearExportDeemedSuppliesDTO":null,"pastYearTableDTO":null,"iecStatusHolderDTO":null,"epcgAuthorisationDetailsVoList":[],"aaAuthorisationDetailsVoList":[],"dfiaAuthorisationVO":[],"rcmcDetailDTOList":null,"industrialDetailDTOList":null,"linkedIECDTOs":null,"lifecycleDTOs":[],"aeoDtlsDTO":null,"insertFlag":null,"updateFlag":null,"deleteFlag":null,"modificationCheck":null,"modificationFileNumber":null,"appId":null,"cibicStatusVal":null,"cibicStatus":null,"declarationCheck":null,"declaration":"","signBy":null,"iecMstId":null,"isIecApplied":null,"iecRole":null,"branchIdIncrement":null,"dirIdIncrement":null,"suspensionDate":null,"cancelDate":null,"iecNo":null,"iecStatusVal":null,"revokeCancellationFlag":false,"authIdIncrement":null,"meisAuthorisationDetailsVoList":null,"rosctlAuthorisationDetailsVoList":null,"gemsearchAuthorisationDetailsVoList":null,"importauthorisationDetailDTOs":null,"scometauthorisationDetailDTOs":null,"exportauthorisationDetailDTOs":null,"searchAuthorisationDetailsVoList":null,"trqsearchAuthorisationDetailsVoList":null,"mobOrEmailModified":null,"mobModified":null,"emailModified":null,"scripManagementDTOs":null,"exportList":null,"panValidated":null,"place":null,"date":null,"declarationDate":null,"id":0,"iecId":0,"panIecNumber":"AALCB5105H","nameOnPan":null,"entityName":null,"panAsIEC":null,"raOffice":null,"dob":null,"cbdtValidated":null,"natureOfConcern":null,"prefferredActivities":null,"iecStatus":null,"modifyReqType":"Update IEC Details","invokeBy":null,"iecFileNumber":null,"iecFileDate":null,"ecomRef":null,"kyc":null,"kycStatus":null,"iecNumeric":null,"iecNumericStatus":null,"fileNumber":null,"iecNumber":null,"riskFlag":null,"iecCertificateDto":null,"applicationNumber":"ARNIECPAPPLY02889471AM24","importList":null,"panInNList":false,"manageProfileFlag":null,"processName":null}'''
det = json.loads(det)
det['applicationNumber']=  application_number

In [452]:
#SUBMISSION OF OTHER DETAILS 
url = f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=handleRequest&screenId=50001&direction=next&currentTab=otherDetails&fromTab=&fromIndex=&toIndex=&_csrf={csrf}"

headers = {
    "Host": "www.dgft.gov.in",
    "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.dgft.gov.in",
    "Referer": "https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=checkLogin",
    }

data = {
    "iecModule": json.dumps(det),
    "draftId": f"{draft_id}",
    
}
response = session.post(url, headers=headers, data=data)
awsalb_cookie = response.cookies.get('AWSALB')   
Jsss = response.cookies.get('JSESSIONID')
print(response.status_code)
print(response.text)



200






	
	
	<input type="hidden" id="currentTabJson" value="{&quot;tabName&quot;:&quot;Other Details (Preferred sectors of operations)&quot;,&quot;viewName&quot;:&quot;otherDetails&quot;,&quot;moduleName&quot;:&quot;otherDetails&quot;,&quot;isVisited&quot;:false}" >
	<input type="hidden" id="moduleJson" value="{&quot;deficiencyDto&quot;:{&quot;application_no&quot;:null,&quot;req_file_no&quot;:null,&quot;deficiency_issued_date&quot;:null,&quot;submited_date&quot;:null,&quot;page&quot;:null,&quot;deficiencyUrl&quot;:null,&quot;req_id&quot;:null,&quot;deficiency&quot;:false,&quot;remarks&quot;:null,&quot;attachedBy&quot;:null,&quot;appId&quot;:201000001},&quot;iecRequestAddressDetailsDTO&quot;:null,&quot;iecRequestBranchDetailsDTOs&quot;:null,&quot;bankDetailsDTOs&quot;:[{&quot;faxnumber&quot;:null,&quot;isNewBankDetails&quot;:null,&quot;isMandatory&quot;:null,&quot;bankNameKeyValueDto&quot;:{&quot;key&quot;:0,&quot;value&quot;:&quot;IDFC BANK&quot;},&quot;bankName&quot;:&quot;IDFC BAN

In [453]:
url = f"https://www.dgft.gov.in/CP/webHP?requestType=ApplicationRH&actionVal=declaration&declaration_temp_id=40&screenId=90000602&_csrf={csrf}"

headers = {
    "Host": "www.dgft.gov.in",
    "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.dgft.gov.in",
    "Referer": "https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=checkLogin",

    
}
data = {
    "temp": ""
}
response = requests.post(url , headers = headers , data = data)
awsalb_cookie = response.cookies.get('AWSALB')
Jsss = response.cookies.get('JSESSIONID')
print(response.status_code)
print(response.text)




200
"<div class=\"bg-white custom-declaration\">\n\t<ul class=\"\">\n\t\t<li>I/We hereby certify that :</li>\n\t\t<ul class=\"declaration-alphabet pl-0\">\n\t\t\t<li>The entity for whom the application has been made have not been penalized\n                under any of the following Acts (as amended from time to time):\n\t\t\t\t<ul>\n\t\t\t\t\t<li>The Customs Act, 1962,</li>\n\t\t\t\t\t<li>The Central Excise Act 1944,</li>\n\t\t\t\t\t<li>Foreign Trade (Development & Regulation) Act 1992, and</li>\n\t\t\t\t\t<li>The Foreign Exchange Management Act,1999;</li>\n\t\t\t\t\t<li>The Conservation of Foreign Exchange, Prevention of\n\t\t\t\t\t\tSmuggling Activities Act, 1974</li>\n\t\t\t\t</ul>\n\t\t\t</li>\n\t\t\t<li>None of the Directors / Partners / Proprietor / Karta /\n\t\t\t\tTrustees of the company /firm /HUF/Trust, (as the case may be),\n\t\t\t\tis/are a Director(s) / Partner(s) / Proprietor / Karta / Trustee in\n\t\t\t\tany other Company/ firm / entity which is on the Denied Entity Lis

In [454]:
url = f"https://www.dgft.gov.in/CP/webHP?requestType=ApplicationRH&actionVal=preview&screenId=90000552&Flag=true&tmpltId=354&cat=203&display=true&_csrf={csrf}"


headers = {
    "Host": "www.dgft.gov.in",
    "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.dgft.gov.in",
    "Referer": "https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=checkLogin",

    
}
data = {
    "portal" : "CAS"
}

response = session.post(url , headers = headers , data = data)
awsalb_cookie = response.cookies.get('AWSALB')   
Jsss = response.cookies.get('JSESSIONID')
print(response.status_code)
print(response.text)


200





<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<meta http-equiv="Content-Type" content="text/html;charset=UTF-8">
<title>Preview</title>
</head>

<body>
<input type="hidden" id="crmCustResultSet" value="" />







<div id="noscript-warning" style="background-color:#b20000;color:white;text-align:center;">
<noscript>
	Note: JavaScript is not enabled! To continue, please enable JavaScript. Check your browser settings to enable JavaScript.
</noscript>
</div>

<input type="hidden" name="OnlyNumAlert" id="OnlyNumAlert" value='Alphabets and special characters are not allowed' />
<input type="hidden" name="SpecialCharAndSpaceAlert" id="SpecialCharAndSpaceAlert" value='Special characters and space are not allowed' />
<input type="hidden" name="allZeroNotAllowed" id="allZeroNotAllowed" value='All zeros or spaces are not allowed.' />
<input type="hidden" name="hdnCmnDateAlert"  id="hdnCmnDateAlert" value='Invalid Date

In [455]:

# Define the POST URL
url = f"https://www.dgft.gov.in/CP/webHP?requestType=ApplicationRH&actionVal=preview&screenId=90000552&Flag=true&tmpltId=354&cat=203&display=true&_csrf={csrf}"

# Define headers (you may need to adjust these headers based on your needs)
headers = {
    "User-Agent": "Your User Agent String",
    "Referer": "https://www.dgft.gov.in/CP/index.jsp",
    # Add any other necessary headers here
   "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.dgft.gov.in",
    "Referer": "https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=checkLogin",

}

# Define any form data (if required)
data = {
    "portal": "CAS",   
}

# Perform the POST request
response = session.post(url, headers=headers, data=data)
awsalb_cookie = response.cookies.get('AWSALB')   
Jsss = response.cookies.get('JSESSIONID')

# Check if the request was successful (status code 200)
print(response.text)






<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<meta http-equiv="Content-Type" content="text/html;charset=UTF-8">
<title>Preview</title>
</head>

<body>
<input type="hidden" id="crmCustResultSet" value="" />







<div id="noscript-warning" style="background-color:#b20000;color:white;text-align:center;">
<noscript>
	Note: JavaScript is not enabled! To continue, please enable JavaScript. Check your browser settings to enable JavaScript.
</noscript>
</div>

<input type="hidden" name="OnlyNumAlert" id="OnlyNumAlert" value='Alphabets and special characters are not allowed' />
<input type="hidden" name="SpecialCharAndSpaceAlert" id="SpecialCharAndSpaceAlert" value='Special characters and space are not allowed' />
<input type="hidden" name="allZeroNotAllowed" id="allZeroNotAllowed" value='All zeros or spaces are not allowed.' />
<input type="hidden" name="hdnCmnDateAlert"  id="hdnCmnDateAlert" value='Invalid Date For

In [456]:
summary = '''{"deficiencyDto":{"application_no":null,"req_file_no":null,"deficiency_issued_date":null,"submited_date":null,"page":null,"deficiencyUrl":null,"req_id":null,"deficiency":false,"remarks":null,"attachedBy":null,"appId":201000001},"iecRequestAddressDetailsDTO":null,"iecRequestBranchDetailsDTOs":null,"bankDetailsDTOs":[{"faxnumber":null,"isNewBankDetails":null,"isMandatory":null,"bankNameKeyValueDto":{"key":1026,"value":"IDFC BANK"},"bankName":"IDFC BANK","branchAddress":"Cunningham","accNumber":"10107749364","accHolder":"BB FINERY","ifsCode":"IDFB0080176","json":null,"bankFlag":null,"bankValidRms":null,"bankAttachmentDTO":{"attachId":null,"s3Path":"2023/09/03/1693743423447_3083913237178366460.tmp* *cancelled cheque.pdf","fileName":"cancelled cheque.pdf","fileType":null,"fileSize":null,"attachmentType":{"key":10000002,"value":"IEC Bank DOC"},"remark":null,"refTableId":null,"pkId":null,"hashDocuments":[{"path":"2023/09/03/1693743423447_3083913237178366460.tmp","name":"cancelled cheque.pdf","hash":"e6c11f876295ce3a7efb2324669d24f399024e77d9a66b814cd85d7ecbe47f5f"}],"attachDate":null,"update":false},"applicantName":null,"applicantPanNo":null,"accType":null,"pfmsValidated":0,"isPrimary":"YES","pfmsStatus":"Not Initiated","bankPkId":0,"dobApplicant":null,"aplicantAddress":null,"applicantAddress1":null,"applicantMobile":null,"applicantEmail":null,"applicantAddress2":null}],"iecRequestBankDetailsDTO":{"faxnumber":null,"isNewBankDetails":null,"isMandatory":1,"bankNameKeyValueDto":{"key":-1,"value":"Please Select"},"bankName":"IDFC BANK","branchAddress":"","accNumber":"","accHolder":"","ifsCode":"","json":null,"bankFlag":null,"bankValidRms":false,"bankAttachmentDTO":{"attachId":null,"s3Path":null,"fileName":null,"fileType":null,"fileSize":null,"attachmentType":null,"remark":null,"refTableId":null,"pkId":null,"hashDocuments":null,"attachDate":null,"update":false},"applicantName":null,"applicantPanNo":null,"accType":null,"pfmsValidated":null,"isPrimary":null,"pfmsStatus":null,"bankPkId":null,"dobApplicant":null,"aplicantAddress":null,"applicantAddress1":null,"applicantMobile":null,"applicantEmail":null,"applicantAddress2":null},"iecRequestOtherDetailsDTO":{"importList":[],"exportList":[{"key":111000157,"value":"Textile & Apparel Products"}],"importServicesList":[],"exportServicesList":[],"service":null,"please_specify":"","isImport":false,"isExport":true,"isOther":false,"isGoods":true,"isServices":false,"importGoods":false,"importServices":false,"exportServices":{"key":0,"value":"-"},"importGoodsOther":false,"importGoodsOtherSpecify":"","exportGoodsOther":false,"exportGoodsOtherSpecify":"","import":false},"iecRequestDirectorDetailsDTOs":[{"siNo":null,"din":"0009730658","dinInt":"9730658","ifForeignDirector":"N","passportNumber":"","directorId":null,"iecDirectorId":null,"panDirector":"CIKPK4752M","fatherName":"SHYAM BIHARI SINGH","birthdateDir":"01/09/1992","addressLine1":"GRAM PAWARA POST","addressLine2":"KUSUMHARA THANA SURYAPURA","city":"Bengaluru","directorState_key":10,"title_key":111000128,"title_Value":"Mr.","directorState_Value":"BIHAR","directorDistrict_Value":"ROHTAS","directorDistrict_key":70,"mcaValidated":"true","title":{"key":111000128,"value":"Mr."},"directorName":"PAWAN KUMAR","directorNameAsPAN":"PAWAN KUMAR","directorDistrict":{"key":70,"value":"ROHTAS"},"directorState":{"key":10,"value":"BIHAR"},"directorPin":"802218","directorMobile":"9284573472","directorDistrictLst":null,"directorStateLst":null},{"siNo":null,"din":"0010113480","dinInt":"10113480","ifForeignDirector":"N","passportNumber":"","directorId":null,"iecDirectorId":null,"panDirector":"NNKPS1275K","fatherName":"SUNIL KUMAR SINGH","birthdateDir":"11/10/2000","addressLine1":"GITA APARTMENTS HOUSE NO 32","addressLine2":"SARDAR PATEL PATH","city":"Bengaluru","directorState_key":10,"title_key":111000127,"title_Value":"Ms.","directorState_Value":"BIHAR","directorDistrict_Value":"PATNA","directorDistrict_key":73,"mcaValidated":"true","title":{"key":111000127,"value":"Ms."},"directorName":"AAYUSHI SINGH","directorNameAsPAN":"AAYUSHI SINGH","directorDistrict":{"key":73,"value":"PATNA"},"directorState":{"key":10,"value":"BIHAR"},"directorPin":"800013","directorMobile":"9823472843","directorDistrictLst":null,"directorStateLst":null}],"iecRequestGeneralDetailsDTO":{"iecPanId":null,"panNumber":"AALCB5105H","nameAsPan":"BB FINERY PRIVATE LIMITED","birthdate":"12/04/2023","addressChangedRms":null,"addMobile":"7050541423","addEmail":"Satvikmishra2615@gmail.com","sezEntity":"NO","eouEntity":"NO","districtLst":null,"stateLst":null,"iecRequestAddressDetailsDTO":null,"iecRequestBranchDetailsDTOs":[{"branchDistrictLst":null,"branchStateLst":null,"branchState":{"key":29,"value":"KARNATAKA"},"branchDistrict":{"key":273,"value":"BENGALURU URBAN"},"branchPinCode":560085,"gstinNumber":null,"isGstinVal":null,"gstinValDate":null,"branchRaOfficeId":null,"branchRaOffice":null,"siNo":1,"iecId":null,"completeAddress":null,"branchCode":1,"iecBranchId":null,"sezBranch":"NO","eouBranch":"NO","gstinBranch":"","branchCity":"Bengaluru","branchDistrict_Value":"BENGALURU URBAN","branchState_Value":"KARNATAKA","branchState_key":29,"gstinValidated":null,"branchId":null,"cinValidated":null,"branchPkId":0,"branchStatus":"N","branchDistrict_key":273,"branchAddressLine1":"BMS COLLEGE OF ENG","branchAddressLine2":""}],"gstinValidated":"","cinValidated":null,"preferredActivity":{"key":111000121,"value":"Merchant Exporter"},"natureOfConcern":{"key":111000113,"value":"Private Limited"},"json":null,"id":0,"cin":"U82990KA2023PTC172277","gstin":"","entityName":"BB FINERY PRIVATE LIMITED","entityAddress":"BMS COLLEGE IF ENG","addressLine2":"","addMobileOTP":null,"addEmailOTP":null,"generalDetailsAttachmentDTO":{"attachId":null,"s3Path":"2023/09/03/1693743023865_8334324162307840963.tmp* *Certificate of Incorporation .pdf*  *0","fileName":null,"fileType":null,"fileSize":null,"attachmentType":{"key":10000000,"value":"IEC General Doc"},"remark":null,"refTableId":null,"pkId":null,"hashDocuments":[{"path":"2023/09/03/1693743023865_8334324162307840963.tmp","name":"Certificate of Incorporation .pdf","hash":"e59dd1f23152d3d5b0b7a0f6ea107bfc5f293824c5c09868f845e1f52b2cdc78"}],"attachDate":null,"update":false},"addressDetailsAttachmentDTO":{"attachId":null,"s3Path":"2023/09/03/1693743065402_1319944290603608945.tmp* *Electricity.pdf*  *0","fileName":null,"fileType":null,"fileSize":null,"attachmentType":{"key":10000001,"value":"IEC Address Doc"},"remark":null,"refTableId":null,"pkId":null,"hashDocuments":[{"path":"2023/09/03/1693743065402_1319944290603608945.tmp","name":"Electricity.pdf","hash":"c574de59f1292944add9c1a42b6795e3b4026a0bcd5f895bd4250fb595555d6f"}],"attachDate":null,"update":false},"dgftOfficeList":null,"adreessline":null,"city":"Bengaluru","state":{"key":29,"value":"KARNATAKA"},"district":{"key":273,"value":"BENGALURU URBAN"},"pincode":"560085","dgft":{"key":11100008,"value":"RA BENGALURU"},"branchCodes":null,"pleaseSpecifyNatureOfConcern":"","pleaseSpecifyPreferred":"","panValidated":null,"isCINChange":true},"isDigitalSigned":null,"paymentId":null,"reqType":"applyForIEC","address2":null,"isPanExist":null,"dscId":null,"appID":null,"gstin":null,"panDirector":"","delOrders":null,"delStatus":null,"birthdateDir":"","directorNameAsPAN":"","isVisited":true,"checkStatus":null,"checkProfile":null,"fileId":null,"manageIecserviceReq":null,"surrenderflag":null,"revokeSuspensionflag":null,"cin":null,"panNumber":null,"suspendedReasons":null,"pastYearExportDeemedSuppliesDTO":null,"pastYearTableDTO":null,"iecStatusHolderDTO":null,"epcgAuthorisationDetailsVoList":[],"aaAuthorisationDetailsVoList":[],"dfiaAuthorisationVO":[],"rcmcDetailDTOList":null,"industrialDetailDTOList":null,"linkedIECDTOs":null,"lifecycleDTOs":[],"aeoDtlsDTO":null,"insertFlag":null,"updateFlag":null,"deleteFlag":null,"modificationCheck":null,"modificationFileNumber":null,"appId":null,"cibicStatusVal":null,"cibicStatus":null,"declarationCheck":"true","declaration":"","signBy":null,"iecMstId":null,"isIecApplied":null,"iecRole":null,"branchIdIncrement":1,"dirIdIncrement":null,"suspensionDate":null,"cancelDate":null,"iecNo":null,"iecStatusVal":null,"revokeCancellationFlag":false,"authIdIncrement":null,"meisAuthorisationDetailsVoList":null,"rosctlAuthorisationDetailsVoList":null,"gemsearchAuthorisationDetailsVoList":null,"importauthorisationDetailDTOs":null,"scometauthorisationDetailDTOs":null,"exportauthorisationDetailDTOs":null,"searchAuthorisationDetailsVoList":null,"trqsearchAuthorisationDetailsVoList":null,"mobOrEmailModified":null,"mobModified":null,"emailModified":null,"scripManagementDTOs":null,"exportList":null,"panValidated":null,"place":"Bangalore","date":null,"declarationDate":"03-09-2023","id":0,"iecId":0,"panIecNumber":"AALCB5105H","nameOnPan":null,"entityName":null,"panAsIEC":null,"raOffice":null,"dob":null,"cbdtValidated":null,"natureOfConcern":null,"prefferredActivities":null,"iecStatus":null,"modifyReqType":"Update PAN Details","invokeBy":null,"iecFileNumber":null,"iecFileDate":null,"ecomRef":null,"kyc":null,"kycStatus":null,"iecNumeric":null,"iecNumericStatus":null,"fileNumber":null,"iecNumber":null,"riskFlag":null,"iecCertificateDto":null,"applicationNumber":"ARNIECPAPPLY02915092AM24","importList":null,"panInNList":false,"manageProfileFlag":null,"processName":null} '''
summary = json.loads(summary)
summary['applicationNumber']=  application_number

In [457]:
url = f"https://www.dgft.gov.in/CP/webHP?requestType=ApplicationRH&actionVal=iecsummary&print=true&moduleName=applyForIEC&screenId=9000012349&dataSubmission=03/09/2023&mpgId=undefined&arn=undefined&_csrf={csrf}"

# Define headers (you may need to adjust these headers based on your needs)
headers = {
    "User-Agent": "Your User Agent String",
    "Referer": "https://www.dgft.gov.in/CP/index.jsp",
    # Add any other necessary headers here
    "Set-Cookie": f"AWSALB={awsalb_cookie}",
    "Set-Cookie": f"JSESSIONID={Jsss}",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.dgft.gov.in",
    "Referer": "https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=checkLogin",

}
data = {
    "summaryjson" : json.dumps(summary)
}

response = session.post(url, headers=headers, data=data)
awsalb_cookie = response.cookies.get('AWSALB')  
Jsss = response.cookies.get('JSESSIONID')
print(response.status_code)
print(response.text)



200






<script>
	var bytes = new Uint8Array([37, 80, 68, 70, 45, 49, 46, 52, 10, 37, -30, -29, -49, -45, 10, 52, 32, 48, 32, 111, 98, 106, 10, 60, 60, 47, 70, 105, 108, 116, 101, 114, 47, 70, 108, 97, 116, 101, 68, 101, 99, 111, 100, 101, 47, 76, 101, 110, 103, 116, 104, 32, 50, 51, 48, 54, 62, 62, 115, 116, 114, 101, 97, 109, 10, 120, -100, -107, 90, 91, 83, -37, 72, 22, 126, -9, -81, -24, 71, 82, 21, 53, 125, -43, 101, 95, 118, 13, 24, -62, -122, 24, 6, -101, -35, -38, -87, -68, 104, 109, 1, -38, -63, 22, 35, -100, -55, -28, -33, -17, 105, 89, 81, -33, 59, -103, 74, 37, 51, -76, -66, -81, -49, -7, -92, -93, 62, 23, -15, -5, -20, 108, 61, -29, 57, 42, 100, -114, -42, -37, 25, -91, 12, 87, 57, -54, 24, -59, -61, -49, -89, -105, 20, -47, 18, -83, 31, 103, 39, 23, 109, -33, 108, 14, 93, 95, 31, 26, 116, -43, -20, -101, -66, 126, 65, -35, 35, -70, -20, -6, -90, 125, -38, -93, 117, 95, 111, -101, 119, -21, -1, -51, -78, 113, 15, -94, -8, 44, 47, -80, 68, 25, 21, -72, 18, -22, -25, 19, 5

In [458]:
pip install PyPDF2


Note: you may need to restart the kernel to use updated packages.


In [459]:
a= response.text
import re

In [460]:
# Define a regular expression to match the binary data
pattern = r'var bytes = new Uint8Array\((\[.*\])\);'

# Use regex to extract the binary data
match = re.search(pattern, a)

if match:
    # Get the binary data from the matched group
    binary_data = match.group(1)

    # Remove unnecessary characters and convert to list of integers
    binary_data = re.sub(r'[^\d,\s-]', '', binary_data)
    binary_data = [int(x) % 256 for x in binary_data.split(',')]

    # Convert the binary data to bytes
    byte_data = bytes(binary_data)

    # Convert bytes to base64 encoding
    base64_data = base64.b64encode(byte_data).decode()

    # Create a data URI for downloading
    data_uri = f"data:application/pdf;base64,{base64_data}"

    # Optionally, you can save the binary data to a PDF file
    with open("output.pdf", "wb") as pdf_file:
        pdf_file.write(byte_data)

    # Now, you can use the 'data_uri' to open the PDF in a browser or download it
    print("Data URI for the PDF:", data_uri)
else:
    print("Binary data not found in the response text.")

Data URI for the PDF: ...


In [181]:
application_number

'ARNIECPAPPLY02915068AM24'